In [1]:
import os
import glob
import re

In [2]:
python_config='/home/ec2-user/DAWN/DJA-SEpp/config/sepp-config.py'
field = 'GDS'

## Detection image
detect_img=glob.glob(f'/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*ir*sci_cutout.fits')[0]
# Filter to use on the detection image
filt='/home/ec2-user/DAWN/DJA-SEpp/config/gauss_3.0_5x5.conv'

## Input images
filter_list = set([re.search('(f\d+\w+)', filename).group(1) for filename in glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*clear*sci*.fits")])
print(filter_list)
args = {'fit_case' : 'sersic_rg4',
        'list_of_IMG_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*{filter}*sci_cutout.fits")[0] for filter in filter_list],
        'list_of_WHT_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*{filter}*wht_cutout.fits")[0] for filter in filter_list],
        'list_of_PSF_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/{field}/*{filter}*star_psf.psf")[0] for filter in filter_list],
        }

## Output
properties='PixelCentroid,WorldCentroid,SourceIDs,GroupInfo,GroupStamp,SourceFlags,NDetectedPixels,NCorePixel,AperturePhotometry,AutoPhotometry,FluxRadius,SNRRatio,ShapeParameters,FlexibleModelFitting'
name = ".".join(detect_img.split("/")[-1].split(".")[:-1])
output_catname=f'/home/ec2-user/DAWN/DJA-SEpp/sepp/{field}/{name}_sepp_cat.fits'
checkimg_path=f'/home/ec2-user/DAWN/DJA-SEpp/sepp/{field}/checkimages'
os.makedirs(checkimg_path, exist_ok=True)


{'f090w', 'f444w', 'f200w', 'f150w', 'f356w', 'f115w', 'f277w'}


In [3]:
os.system(f"sourcextractor++ \
         --python-config-file {python_config} \
         --python-arg '{args}' \
         --output-catalog-filename {output_catname} \
         --output-properties {properties} \
         --output-flush-size 10 \
         --detection-image {detect_img} \
         --weight-type WEIGHT \
         --weight-absolute 0 \
         --segmentation-filter {filt} \
         --background-cell-size 128 \
         --segmentation-algorithm LUTZ \
         --smoothing-box-size 5 \
         --detection-threshold 1.15 \
         --detection-minimum-area 6 \
         --partition-corethreshold yes \
         --core-threshold-value 2.25 \
         --core-minimum-area 15 \
         --grouping-algorithm split \
         --partition-multithreshold yes \
         --partition-minimum-area 18 \
         --grouping-hard-limit 0 \
         --partition-minimum-contrast 0.0001 \
         --partition-threshold-count 42 \
         --use-cleaning yes \
         --cleaning-minimum-area 15 \
         --model-fitting-iterations 550 \
         --sampling-scale-factor 1 \
         --check-image-model-fitting {checkimg_path}/model.fits \
         --check-image-residual {checkimg_path}/resid.fits \
         --log-file {output_catname}_assoc.log \
         --log-level INFO \
         ")


2024-05-02T06:02:20UTC Config  INFO : Loaded segmentation filter: /home/ec2-user/DAWN/DJA-SEpp/config/gauss_3.0_5x5.conv width: 5 height: 5
2024-05-02T06:02:20UTC Python::stderr  INFO : /home/ec2-user/miniconda3/envs/dawn-310/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
2024-05-02T06:02:20UTC Python::stderr  INFO :   setattr(self, word, getattr(machar, word).flat[0])
2024-05-02T06:02:20UTC Python::stderr  INFO : /home/ec2-user/miniconda3/envs/dawn-310/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
2024-05-02T06:02:20UTC Python::stderr  INFO :   return self._float_to_str(self.smallest_subnormal)
2024-05-02T06:02:20UTC Python::stdout  INFO : F090W
2024-05-02T06:02:20UTC Python::stdout  INFO : F444W
2024-05-02T06:02:20UTC Python::stdout  INFO : F200W
2024-05-02T06:02:20UTC Python::stdout